In [21]:
# good practice is to get the code working and then make it fast
# ipython has some magic commands that can help with this
# %time and %timeit are two of them to time code
# %prun is another one to profile code along with %lprun and %mprun for line-by-line and memory profiling
# %memit is another one to measure the memory use of a single statement
%timeit sum(range(100))

899 ns ± 47.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [22]:
# timeit runs the code several times to get a good estimate of the run time
def func3():
    total = 0
    for i in range(1000):
        for j in range(1000):
            total += i * (-1) ** j
        
%timeit func3()

145 ms ± 4.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

660 µs ± 34.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [24]:
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 15.4 ms, sys: 127 µs, total: 15.5 ms
Wall time: 15.6 ms


In [25]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 1.37 ms, sys: 32 µs, total: 1.4 ms
Wall time: 1.44 ms


In [26]:
%time func3()

CPU times: user 145 ms, sys: 2.8 ms, total: 148 ms
Wall time: 150 ms


In [27]:
# statements in context is sometimes more important than the time it takes to run a single statement for this we use %prun
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

In [28]:
%prun sum_of_lists(1000000)

         14 function calls in 0.841 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.761    0.152    0.761    0.152 3077943115.py:5(<listcomp>)
        1    0.035    0.035    0.830    0.830 3077943115.py:2(sum_of_lists)
        5    0.034    0.007    0.034    0.007 {built-in method builtins.sum}
        1    0.011    0.011    0.841    0.841 <string>:1(<module>)
        1    0.000    0.000    0.841    0.841 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [31]:
# can do line by line profiling with %lprun
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [30]:
%lprun -f sum_of_lists sum_of_lists(5000)

Timer unit: 1e-09 s

Total time: 0.00665 s
File: /var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_57111/3077943115.py
Function: sum_of_lists at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def sum_of_lists(N):
     3         1       1000.0   1000.0      0.0      total = 0
     4         5       3000.0    600.0      0.0      for i in range(5):
     5         5    6440000.0 1288000.0     96.8          L = [j ^ (j >> i) for j in range(N)]
     6         5     205000.0  41000.0      3.1          total += sum(L)
     7         1       1000.0   1000.0      0.0      return total

In [32]:
%memit sum_of_lists(1000000)

peak memory: 189.97 MiB, increment: 84.81 MiB


In [33]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Writing mprun_demo.py


In [34]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

Filename: /Users/russell/Documents/Code/Python/Data_Science/mprun_demo.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1    112.4 MiB    112.4 MiB           1   def sum_of_lists(N):
     2    112.4 MiB      0.0 MiB           1       total = 0
     3    125.9 MiB      0.0 MiB           6       for i in range(5):
     4    159.6 MiB -49665594.8 MiB     5000015           L = [j ^ (j >> i) for j in range(N)]
     5    159.6 MiB      0.0 MiB           5           total += sum(L)
     6    125.9 MiB   -126.9 MiB           5           del L # remove reference to L
     7    125.9 MiB      0.0 MiB           1       return total